In [1]:
import csv
import sys
from elasticsearch import Elasticsearch, helpers
import datetime, time
# import codecs

class ElasticImporter():
    def __init__(self,elastic_url=None, batch_size=10):
        self.elastic_url = elastic_url
        self.batch_size = batch_size
        self.es = Elasticsearch()
        reload(sys)
        sys.setdefaultencoding('utf-8')
        
    def import_file(self,file_path, index, doc_type, id_column):
        with open(file_path, 'rb') as csvfile:
            reader = csv.reader(csvfile,quotechar='"')
            batch_count = 0
            rows = []
            for idx, row in enumerate(reader):
                if idx == 0:
                    header = row
                    continue
                
                rows.append(row)
                if len(rows) == self.batch_size:
                    self.import_to_es(header, rows, index, doc_type,id_column)
                    rows = []
                
                if idx%1000 == 0: 
                    print 'imported {0} rows for index {1}'.format(idx, index)
                    

    def import_to_es(self, header, rows, index, doc_type, id_column):
        actions = []
        for row in rows:
            action = dict(zip(header, row))
            action['_index'] = index
            action['_type'] = doc_type
            action['_id'] = action.pop(id_column)
            actions.append(action)
        helpers.bulk(self.es, actions)
    
importer = ElasticImporter()
importer.import_file('/Users/ajaykumar.c/projects/github/elasticsearch_utils/srcm_group.csv','srcm_group','external')
# es = Elasticsearch()
# es.indices.create(index='test-index', ignore=400)


In [31]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%perl  %%prun  %%pypy  %%python  %%python2  %%python3